In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse, r2_score as R2
from sklearn.model_selection import train_test_split

In [20]:
device = torch.device("cpu")
dtype = torch.double

# #1) SCAD PyTorch Class

## Implementation

In [70]:
class SCAD(nn.Module):
    def __init__(self, input_size, alpha=3.7, lambda_val=0.5):
      '''
      Initialize the SCAD Regression Model.
      Default values are derived from JSTOR paper example.
      Args:
        - input_size (int) : Number of input features.
        - alpha (float) : Hyperparameter that determines how quickly penalty declines as B increases
        - lambda_val (float) : Hyperparam that determines B value threshold limits and corresponding penalty
      '''
      # Initializing the parent class
      super(SCAD, self).__init__()
      # Assigning params
      self.input_size = input_size
      self.alpha = alpha
      self.lambda_val = lambda_val
      # Defining linear regression layer w/o bias
      self.linear = nn.Linear(input_size, 1, bias=False, device=device, dtype=dtype)

      # Linear combination of the feature values
      # Compute the linear combination for a given set of weights
    def forward(self, x):
      '''
      Forward pass of the SCAD model.

      Args:
        x (Tensor): Input data with shape (batch_size, input_size)/

      Returns:
        Tensor: Predicted values with shape (batch_size, 1).
      '''
      # Do the linear combination
      return self.linear(x)

    def loss(self, y_pred, y_true):
      '''
      Compute the SCAD loss function

      Args:
        y_pred (Tensor): Predicted values with shape (batch_size, 1).
        y_true (Tensor): True target values with shape (batch_size, 1).

      Returns:
        Tensor: The SCAD Loss.
      '''

      # Baseline MSE to add additional penalties to
      mse_loss = nn.MSELoss()(y_pred, y_true)

      weights = self.linear.weight

      # Referencing https://andrewcharlesjones.github.io/journal/scad.html
      # Depending on the values of the weights in regards to alpha and lambda, determine their associated penalties
      is_linear = (torch.abs(weights) <= self.lambda_val)
      is_quadratic = torch.logical_and(self.lambda_val < torch.abs(weights), torch.abs(weights) <= self.alpha * self.lambda_val)
      is_constant = (self.alpha * self.lambda_val) < torch.abs(weights)

      linear_part = (self.lambda_val * torch.abs(weights) * is_linear).sum()
      quadratic_part = ((2 * self.alpha * self.lambda_val * torch.abs(weights) - weights**2 - self.lambda_val**2) / (2 * (self.alpha - 1)) * is_quadratic).sum()
      constant_part = ((self.lambda_val**2 * (self.alpha + 1)) / 2 * is_constant).sum()

      # The sum of the MSE + SCAD weight penalties
      return mse_loss + linear_part + quadratic_part + constant_part


    def fit(self, X, y, num_epochs=100, learning_rate=0.001):
      '''
      Fit the SCAD model to the training data.

      Args:
        X (Tensor): Input data w/ shape (num_samples, input_size).
        y (Tensor): Target values with shape (num_samples, 1).
        num_epochs (int): Number of training epochs.
        learning_rate (float): Learning rate for optimization.
      '''
      optimizer = optim.SGD(self.parameters(), lr=learning_rate)
      # Instead of SGD you could use Adaptive Descent => optim.Adam(self.parameters(), lr=learning_rate)

      for epoch in range(num_epochs):
          self.train()
          optimizer.zero_grad()
          y_pred = self(X)
          # Prevent number contam
          loss = self.loss(y_pred.flatten(), y.flatten())
          # Updating the weights in the direction of the negative gradient w/ optimizer
          # Compute new gradient
          loss.backward()
          # Update weights in direction of negative gradient
          optimizer.step()

          if (epoch + 1) % 1000 == 0:
              print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}")

    def predict(self, X):
      """
      Predict target values for input data.

      Args:
        X (Tensor): Input data w/ shape (num_samples, input_size).

      Returns:
        Tensor: Predicted values w/ shape (num_samples, 1).
      """
      self.eval()
      with torch.no_grad():
        y_pred = self(X)
      return y_pred

    def get_coefficients(self):
        """
        Get the coefficients (weights) of the linear regression layer.

        Returns:
            Tensor: Coefficients with shape (output_size, input_size).

        """

        return self.linear.weight


## Implementation With real data & Variable Selection

In [71]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
scaler = MinMaxScaler()

### Running with real data (concrete dataset)

In [72]:
data = pd.read_csv('/content/drive/MyDrive/ML2023/data/concrete.csv')
x = data.drop(columns=['strength']).values
y = data["strength"].values
x = scaler.fit_transform(x)
x_torch = torch.tensor(x, device=device)
y_torch = torch.tensor(y, device=device)
model_scad = SCAD(input_size=x_torch.shape[1], alpha=3.7, lambda_val=0.5)
model_scad.fit(x_torch, y_torch, num_epochs=10000, learning_rate=0.03)

Epoch [1000/10000], Loss: 119.90800985910722
Epoch [2000/10000], Loss: 114.37446939086723
Epoch [3000/10000], Loss: 112.92896028077199
Epoch [4000/10000], Loss: 112.34706227375979
Epoch [5000/10000], Loss: 112.09798192997035
Epoch [6000/10000], Loss: 111.99058176163291
Epoch [7000/10000], Loss: 111.94423300190888
Epoch [8000/10000], Loss: 111.92422914485218
Epoch [9000/10000], Loss: 111.91559549815906
Epoch [10000/10000], Loss: 111.9118692191612


### Variable Selection

In [73]:
model_scad.get_coefficients()

Parameter containing:
tensor([[ 51.1523,  36.0417,  16.7259, -20.0369,   9.5241,   5.2548,   6.5980,
          41.4736]], dtype=torch.float64, requires_grad=True)

In [74]:
data.head()

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


Based on the $\beta$ values that have the highest weights, SCAD has determined that $X_0$, $X_1$ and $X_7$ are the features with the greatest importance in determining cement strength. These features are respectively, the cement value, the slag value and the age value.

In [75]:
data = pd.read_csv('/content/drive/MyDrive/ML2023/data/concrete.csv')
x = data.drop(columns=['strength']).values
y = data["strength"].values
x = scaler.fit_transform(x)
x_torch = torch.tensor(x, device=device)
y_torch = torch.tensor(y, device=device)
X_train, X_test, y_train, y_test = train_test_split(x_torch, y_torch, test_size=0.33, random_state=42)
model_scad = SCAD(input_size=X_train.shape[1], alpha=3.7, lambda_val=0.5)
model_scad.fit(X_train, y_train, num_epochs=10000, learning_rate=0.03)

Epoch [1000/10000], Loss: 117.94888529616418
Epoch [2000/10000], Loss: 112.77649898280517
Epoch [3000/10000], Loss: 111.5353512831315
Epoch [4000/10000], Loss: 111.05189130080291
Epoch [5000/10000], Loss: 110.84769117344678
Epoch [6000/10000], Loss: 110.76049540917815
Epoch [7000/10000], Loss: 110.7232091532624
Epoch [8000/10000], Loss: 110.7072620732839
Epoch [9000/10000], Loss: 110.70044145527584
Epoch [10000/10000], Loss: 110.69752424598259


In [76]:
mse(model_scad.predict(X_test), y_test)

111.69770283817272

### ElasticNet comparision

In [82]:
class ElasticNet(nn.Module):
    def __init__(self, input_size, alpha=1.0, l1_ratio=0.5):
        """
        Initialize the ElasticNet regression model.

        Args:
            input_size (int): Number of input features.
            alpha (float): Regularization strength. Higher values of alpha
                emphasize L1 regularization, while lower values emphasize L2 regularization.
            l1_ratio (float): The ratio of L1 regularization to the total
                regularization (L1 + L2). It should be between 0 and 1.

        """
        super(ElasticNet, self).__init__()
        # nueron needs to know input size => the number of features you have
        # e.g num of pixels you have
        self.input_size = input_size
        self.alpha = alpha
        self.l1_ratio = l1_ratio

        # Define the linear regression layer
        # ,1 means deal w/ it as 1d area, double() => as double precision
        self.linear = nn.Linear(input_size, 1).double()
        # Forces neuron to ignore the bias - handle intercept term seperately
        # Best practice
        self.linear = nn.Linear(input_size, 1, bias=False, device=device, dtype=dtype)

    # Linear combination of the feature values
    # Compute the linear combination for a given set of weights
    def forward(self, x):
        """
        Forward pass of the ElasticNet model.

        Args:
            x (Tensor): Input data with shape (batch_size, input_size).

        Returns:
            Tensor: Predicted values with shape (batch_size, 1).

        """
        # Do the linear combinations according to this specific combination
        return self.linear(x)

    def loss(self, y_pred, y_true):
        """
        Compute the ElasticNet loss function.

        Args:
            y_pred (Tensor): Predicted values with shape (batch_size, 1).
            y_true (Tensor): True target values with shape (batch_size, 1).

        Returns:
            Tensor: The ElasticNet loss.

        """
        mse_loss = nn.MSELoss()(y_pred, y_true)
        # Lasso
        l1_reg = torch.norm(self.linear.weight, p=1)
        # Ridge
        l2_reg = torch.norm(self.linear.weight, p=2)

        loss = 0.5*mse_loss + self.alpha * (
            0.5*self.l1_ratio * l1_reg + (1 - self.l1_ratio) * (1/2)*l2_reg**2
        )

        return loss

    def fit(self, X, y, num_epochs=100, learning_rate=0.01):
        """
        Fit the ElasticNet model to the training data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).
            y (Tensor): Target values with shape (num_samples, 1).
            num_epochs (int): Number of training epochs.
            learning_rate (float): Learning rate for optimization.

        """
        optimizer = optim.SGD(self.parameters(), lr=learning_rate)
        # Instead of SGD you could use Adaptive Descent => optim.Adam(self.parameters(), lr=learning_rate)

        for epoch in range(num_epochs):
            self.train()
            optimizer.zero_grad()
            y_pred = self(X)
            # Prevent number contam
            loss = self.loss(y_pred.flatten(), y.flatten())
            # Updating the weights in the direction of the negative gradient w/ optimizer
            # Compute new gradient
            loss.backward()
            # Update weights in direction of negative gradient
            optimizer.step()


    def predict(self, X):
        """
        Predict target values for input data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).

        Returns:
            Tensor: Predicted values with shape (num_samples, 1).

        """
        self.eval()
        with torch.no_grad():
            y_pred = self(X)
        return y_pred
    def get_coefficients(self):
        """
        Get the coefficients (weights) of the linear regression layer.

        Returns:
            Tensor: Coefficients with shape (output_size, input_size).

        """

        # Each class has helpful items
        return self.linear.weight


In [85]:
data = pd.read_csv('/content/drive/MyDrive/ML2023/data/concrete.csv')
x = data.drop(columns=['strength']).values
y = data["strength"].values
x = scaler.fit_transform(x)
x_torch = torch.tensor(x, device=device)
y_torch = torch.tensor(y, device=device)
X_train, X_test, y_train, y_test = train_test_split(x_torch, y_torch, test_size=0.33, random_state=42)
model_e = ElasticNet(input_size=X_train.shape[1])
model_e.fit(X_train, y_train, num_epochs=10000, learning_rate=0.001)

In [86]:
mse(model_e.predict(X_test), y_test)

374.1414255710245

# 2) Betastar Simulations

### Functionality to make correlated dataset

In [48]:
import numpy as np
from scipy.linalg import toeplitz

In [49]:
# From lecture "Variable selection and regularization"
def make_correlated_features(num_samples,p,rho):
  vcor = []
  for i in range(p):
    vcor.append(rho**i)
  r = toeplitz(vcor)
  mu = np.repeat(0,p)
  # mu vector of the means
  x = np.random.multivariate_normal(mu, r, size=num_samples)
  return x

def make_correlated_dataset():
  rho =0.9
  p = 200
  n = 150

  x = make_correlated_features(n,p,rho)
  # Simulating a sparsity pattern
  beta =np.array([0,1,2,-3,0,0,0,1,4,2,0,1,-1])
  # Making this sparsity pattern match the number of variables in X (filling in unused vars w/ 0)
  beta = beta.reshape(-1,1)
  betastar = np.concatenate([beta,np.repeat(0,p-len(beta)).reshape(-1,1)],axis=0)
  # Creating a y which utilizes this sparsity pattern plus some random noise
  y = x@betastar + 0.5*np.random.normal(size=(150,1))
  return x,y

### Defining regressors without print statements

In [50]:
class ElasticNet(nn.Module):
    def __init__(self, input_size, alpha=1.0, l1_ratio=0.5):
        """
        Initialize the ElasticNet regression model.

        Args:
            input_size (int): Number of input features.
            alpha (float): Regularization strength. Higher values of alpha
                emphasize L1 regularization, while lower values emphasize L2 regularization.
            l1_ratio (float): The ratio of L1 regularization to the total
                regularization (L1 + L2). It should be between 0 and 1.

        """
        super(ElasticNet, self).__init__()
        # nueron needs to know input size => the number of features you have
        # e.g num of pixels you have
        self.input_size = input_size
        self.alpha = alpha
        self.l1_ratio = l1_ratio

        # Define the linear regression layer
        # ,1 means deal w/ it as 1d area, double() => as double precision
        self.linear = nn.Linear(input_size, 1).double()
        # Forces neuron to ignore the bias - handle intercept term seperately
        # Best practice
        self.linear = nn.Linear(input_size, 1, bias=False, device=device, dtype=dtype)

    # Linear combination of the feature values
    # Compute the linear combination for a given set of weights
    def forward(self, x):
        """
        Forward pass of the ElasticNet model.

        Args:
            x (Tensor): Input data with shape (batch_size, input_size).

        Returns:
            Tensor: Predicted values with shape (batch_size, 1).

        """
        # Do the linear combinations according to this specific combination
        return self.linear(x)

    def loss(self, y_pred, y_true):
        """
        Compute the ElasticNet loss function.

        Args:
            y_pred (Tensor): Predicted values with shape (batch_size, 1).
            y_true (Tensor): True target values with shape (batch_size, 1).

        Returns:
            Tensor: The ElasticNet loss.

        """
        mse_loss = nn.MSELoss()(y_pred, y_true)
        # Lasso
        l1_reg = torch.norm(self.linear.weight, p=1)
        # Ridge
        l2_reg = torch.norm(self.linear.weight, p=2)

        loss = 0.5*mse_loss + self.alpha * (
            0.5*self.l1_ratio * l1_reg + (1 - self.l1_ratio) * (1/2)*l2_reg**2
        )

        return loss

    def fit(self, X, y, num_epochs=100, learning_rate=0.01):
        """
        Fit the ElasticNet model to the training data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).
            y (Tensor): Target values with shape (num_samples, 1).
            num_epochs (int): Number of training epochs.
            learning_rate (float): Learning rate for optimization.

        """
        optimizer = optim.SGD(self.parameters(), lr=learning_rate)
        # Instead of SGD you could use Adaptive Descent => optim.Adam(self.parameters(), lr=learning_rate)

        for epoch in range(num_epochs):
            self.train()
            optimizer.zero_grad()
            y_pred = self(X)
            # Prevent number contam
            loss = self.loss(y_pred, y)
            # Updating the weights in the direction of the negative gradient w/ optimizer
            # Compute new gradient
            loss.backward()
            # Update weights in direction of negative gradient
            optimizer.step()


    def predict(self, X):
        """
        Predict target values for input data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).

        Returns:
            Tensor: Predicted values with shape (num_samples, 1).

        """
        self.eval()
        with torch.no_grad():
            y_pred = self(X)
        return y_pred
    def get_coefficients(self):
        """
        Get the coefficients (weights) of the linear regression layer.

        Returns:
            Tensor: Coefficients with shape (output_size, input_size).

        """

        # Each class has helpful items
        return self.linear.weight


In [51]:
# From lecture
# we can call this version PED_Adam because we use the adaptive momentum gradient descent for optimization
class sqrtLasso(nn.Module):
    def __init__(self, input_size, alpha=0.1):
        """
        Initialize the  regression model.


        """
        super(sqrtLasso, self).__init__()
        self.input_size = input_size
        self.alpha = alpha


        # Define the linear regression layer
        self.linear = nn.Linear(input_size, 1).double()

    def forward(self, x):
        """
        Forward pass of the model.

        Args:
            x (Tensor): Input data with shape (batch_size, input_size).

        Returns:
            Tensor: Predicted values with shape (batch_size, 1).

        """
        return self.linear(x)

    def loss(self, y_pred, y_true):
        """
        Compute the loss function.

        Args:
            y_pred (Tensor): Predicted values with shape (batch_size, 1).
            y_true (Tensor): True target values with shape (batch_size, 1).

        Returns:
            Tensor: The loss.

        """
        mse_loss = nn.MSELoss()(y_pred, y_true)
        l1_reg = torch.norm(self.linear.weight, p=1,dtype=torch.float64)
        # l2_reg = torch.norm(self.linear.weight, p=2,dtype=torch.float64)

        loss = (len(y_true)*mse_loss)**(1/2) + self.alpha * (l1_reg)

        return loss

    def fit(self, X, y, num_epochs=200, learning_rate=0.01):
        """
        Fit the model to the training data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).
            y (Tensor): Target values with shape (num_samples, 1).
            num_epochs (int): Number of training epochs.
            learning_rate (float): Learning rate for optimization.

        """
        optimizer = optim.Adam(self.parameters(), lr=learning_rate)

        for epoch in range(num_epochs):
            self.train()
            optimizer.zero_grad()
            y_pred = self(X)
            loss = self.loss(y_pred, y)
            loss.backward()
            optimizer.step()

    def predict(self, X):
        """
        Predict target values for input data.

        Args:
            X (Tensor): Input data with shape (num_samples, input_size).

        Returns:
            Tensor: Predicted values with shape (num_samples, 1).

        """
        self.eval()
        with torch.no_grad():
            y_pred = self(X)
        return y_pred
    def get_coefficients(self):
        """
        Get the coefficients (weights) of the linear regression layer.

        Returns:
            Tensor: Coefficients with shape (output_size, input_size).

        """
        return self.linear.weight

In [52]:
class SCAD(nn.Module):
    def __init__(self, input_size, alpha=3.7, lambda_val=0.5):
      '''
      Initialize the SCAD Regression Model.
      Default values are derived from JSTOR paper example.
      Args:
        - input_size (int) : Number of input features.
        - alpha (float) : Hyperparameter that determines how quickly penalty declines as B increases
        - lambda_val (float) : Hyperparam that determines B value threshold limits and corresponding penalty
      '''
      # Initializing the parent class
      super(SCAD, self).__init__()
      # Assigning params
      self.input_size = input_size
      self.alpha = alpha
      self.lambda_val = lambda_val
      # Defining linear regression layer w/o bias
      self.linear = nn.Linear(input_size, 1, bias=False, device=device, dtype=dtype)

      # Linear combination of the feature values
      # Compute the linear combination for a given set of weights
    def forward(self, x):
      '''
      Forward pass of the SCAD model.

      Args:
        x (Tensor): Input data with shape (batch_size, input_size)/

      Returns:
        Tensor: Predicted values with shape (batch_size, 1).
      '''
      # Do the linear combination
      return self.linear(x)

    def loss(self, y_pred, y_true):
      '''
      Compute the SCAD loss function

      Args:
        y_pred (Tensor): Predicted values with shape (batch_size, 1).
        y_true (Tensor): True target values with shape (batch_size, 1).

      Returns:
        Tensor: The SCAD Loss.
      '''

      mse_loss = nn.MSELoss()(y_pred, y_true)
      weights = self.linear.weight

      # Using
      is_linear = (torch.abs(weights) <= self.lambda_val)
      is_quadratic = torch.logical_and(self.lambda_val < torch.abs(weights), torch.abs(weights) <= self.alpha * self.lambda_val)
      is_constant = (self.alpha * self.lambda_val) < torch.abs(weights)

      linear_part = (self.lambda_val * torch.abs(weights) * is_linear).sum()
      quadratic_part = ((2 * self.alpha * self.lambda_val * torch.abs(weights) - weights**2 - self.lambda_val**2) / (2 * (self.alpha - 1)) * is_quadratic).sum()
      constant_part = ((self.lambda_val**2 * (self.alpha + 1)) / 2 * is_constant).sum()

      return mse_loss + linear_part + quadratic_part + constant_part


    def fit(self, X, y, num_epochs=100, learning_rate=0.001):
      '''
      Fit the SCAD model to the training data.

      Args:
        X (Tensor): Input data w/ shape (num_samples, input_size).
        y (Tensor): Target values with shape (num_samples, 1).
        num_epochs (int): Number of training epochs.
        learning_rate (float): Learning rate for optimization.
      '''
      optimizer = optim.SGD(self.parameters(), lr=learning_rate)
      # Instead of SGD you could use Adaptive Descent => optim.Adam(self.parameters(), lr=learning_rate)

      for epoch in range(num_epochs):
          self.train()
          optimizer.zero_grad()
          y_pred = self(X)
          # Prevent number contam
          loss = self.loss(y_pred.flatten(), y.flatten())
          # Updating the weights in the direction of the negative gradient w/ optimizer
          # Compute new gradient
          loss.backward()
          # Update weights in direction of negative gradient
          optimizer.step()

    def predict(self, X):
      """
      Predict target values for input data.

      Args:
        X (Tensor): Input data w/ shape (num_samples, input_size).

      Returns:
        Tensor: Predicted values w/ shape (num_samples, 1).
      """
      self.eval()
      with torch.no_grad():
        y_pred = self(X)
      return y_pred

    def get_coefficients(self):
        """
        Get the coefficients (weights) of the linear regression layer.

        Returns:
            Tensor: Coefficients with shape (output_size, input_size).

        """

        return self.linear.weight


In [53]:
from tqdm import tqdm

### Experiment

In [57]:
p = 200
# Simulating a sparsity pattern
beta =np.array([0,1,2,-3,0,0,0,1,4,2,0,1,-1])
# Making this sparsity pattern match the number of variables in X (filling in unused vars w/ 0)
beta = beta.reshape(-1,1)
betastar = np.concatenate([beta,np.repeat(0,p-len(beta)).reshape(-1,1)],axis=0)

In [59]:
scad_difs = []
elastic_difs = []
sqrtlasso_difs = []

for iter in tqdm(range(500)):
  x,y = make_correlated_dataset()
  x_scaled = scaler.fit_transform(x)
  x_torch = torch.tensor(x_scaled, device=device)
  y_torch = torch.tensor(y, device=device)
  model_scad = SCAD(x_torch.shape[1])
  model_elastic = ElasticNet(x_torch.shape[1])
  model_sqrtlasso = sqrtLasso(x_torch.shape[1])

  model_scad.fit(x_torch, y_torch)
  model_elastic.fit(x_torch, y_torch)
  model_sqrtlasso.fit(x_torch, y_torch)

  scad_coefs = model_scad.get_coefficients().cpu().detach().numpy()
  elastic_coefs = model_elastic.get_coefficients().cpu().detach().numpy()
  sqrtlasso_coefs = model_sqrtlasso.get_coefficients().cpu().detach().numpy()

  scad_difs.append(mse(betastar, scad_coefs.reshape(-1,1)))
  elastic_difs.append(mse(betastar, elastic_coefs.reshape(-1,1)))
  sqrtlasso_difs.append(mse(betastar, sqrtlasso_coefs.reshape(-1,1)))

print(" ")
print("Mean scad MSE with regards to betastar is ", np.mean(scad_difs))
print("Mean elastic MSE with regards to betastar is ", np.mean(elastic_difs))
print("Mean sqrt lasso MSE with regards to betastar is ", np.mean(sqrtlasso_difs))


100%|██████████| 500/500 [03:40<00:00,  2.26it/s]

 
Mean scad MSE with regards to betastar is  0.17427232220926658
Mean elastic MSE with regards to betastar is  0.15646798274095441
Mean sqrt lasso MSE with regards to betastar is  0.4115441597152536


It looks like generally ElasticNet produces the coefficients closest to Betastar, with SCAD closely following it, and SQRTLasso last.